In [17]:
from numpy.random import randint
from numpy.random import rand
from math import cos

# objective function
def objective(x):
	return x**3.0 * cos(x**2.0)

def decode(bounds, n_bits, bitstring):
    largest = 2**n_bits
    # convert bitstring to a string of chars
    chars = ''.join([str(s) for s in bitstring])
    integer = int(chars, 2)
    # scale integer to desired range
    value = bounds[0] + (integer/largest) * (bounds[1] - bounds[0])
    return value

# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print("gen -> %d | new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			for c in crossover(p1, p2, r_cross):
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

# define range for input
bounds = [-2.0, 2.0]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)

best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

gen -> 0 | new best f(1.9051513671875) = -6.107746
gen -> 0 | new best f(1.8653564453125) = -6.123451
gen -> 1 | new best f(1.86553955078125) = -6.123783
gen -> 1 | new best f(1.87005615234375) = -6.130836
gen -> 2 | new best f(1.88323974609375) = -6.138763
gen -> 4 | new best f(1.883056640625) = -6.138785
gen -> 7 | new best f(1.881103515625) = -6.138789
gen -> 7 | new best f(1.8814697265625) = -6.138821
gen -> 8 | new best f(1.88201904296875) = -6.138841
gen -> 12 | new best f(1.882080078125) = -6.138841
Done!
f(1.882080078125) = -6.138841
